# Lab 9.2: Brain Size and Intelligence Calculator
## Chapter 9: The Information Revolution - Nervous Systems

### 🎯 Learning Objectives
- Calculate and interpret Encephalization Quotient (EQ)
- Compare brain/body size relationships across vertebrates
- Analyze patterns in brain evolution
- Explore the relationship between brain size and cognitive abilities

### 📖 Connection to Chapter 9
This lab integrates concepts from **Section 9.3: Brain Evolution**:
- Allometric scaling: brain size scales with body size
- Encephalization Quotient (EQ) as measure of relative brain size
- Brain-body size relationships across vertebrates
- Regional brain specialization and cognitive capabilities

### 🧠 The Big Question
**Why do some animals seem "smarter" than others?**

Is it just about brain size? A blue whale has a 9kg brain, while a crow has a 7g brain - yet crows make tools and solve complex problems. What matters more: absolute size or relative size?

In [ ]:
# === GOOGLE COLAB SETUP - RUN THIS FIRST ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled for Google Colab")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, FloatSlider, Dropdown, Button, Output, HTML, IntSlider, HBox, VBox
from IPython.display import display, clear_output
from datetime import datetime
import os

print("✓ All libraries loaded!")
print("Ready to explore brain evolution...")

## Part 1: The Encephalization Quotient Formula

### The Problem
Larger animals naturally have larger brains just to control their bigger bodies.
- Elephant brain: 5,000g
- Human brain: 1,350g
- Crow brain: 7.5g

But crows can solve problems elephants can't! How do we measure "extra" brain capacity?

### The Solution: Encephalization Quotient (EQ)
Compare actual brain size to *expected* brain size for that body size.

$$EQ = \frac{\text{Actual Brain Mass}}{\text{Expected Brain Mass}}$$

Where: $\text{Expected Brain} = 0.12 \times (\text{Body Mass})^{0.67}$

### Interpretation
- **EQ = 1.0**: Average brain for body size
- **EQ > 1.0**: Larger brain than expected (more cognitive capacity)
- **EQ < 1.0**: Smaller brain than expected (specialized for efficiency)

### From Chapter 9 (Table 9.2)
| Group | Average EQ | Range |
|-------|------------|-------|
| Mammals | 0.40-7.50 | Variable |
| Birds | 0.80-1.50 | Consistently high |
| Reptiles | 0.25-0.50 | Lower |
| Amphibians | 0.20-0.35 | Lower |
| Fish | 0.15-0.30 | Lowest |

In [ ]:
# Core EQ calculation function
def calculate_eq(brain_mass_g, body_mass_kg):
    """
    Calculate Encephalization Quotient using Jerison's formula
    
    Parameters:
    - brain_mass_g: Brain mass in grams
    - body_mass_kg: Body mass in kilograms
    
    Returns:
    - eq: Encephalization quotient
    - expected_brain_g: Expected brain mass in grams
    """
    # Jerison's formula for expected brain mass
    # E = 0.12 × M^0.67 (where E is in kg, M is in kg)
    expected_brain_kg = 0.12 * (body_mass_kg ** 0.67)
    expected_brain_g = expected_brain_kg * 1000
    
    # Calculate EQ
    eq = brain_mass_g / expected_brain_g
    
    return eq, expected_brain_g

# Test with human
print("EXAMPLE: HUMAN")
print("=" * 50)
human_brain = 1350  # grams
human_body = 70     # kg

eq, expected = calculate_eq(human_brain, human_body)

print(f"Body mass: {human_body} kg")
print(f"Brain mass: {human_brain} g")
print(f"Expected brain: {expected:.1f} g")
print(f"EQ: {eq:.2f}")
print(f"\n➜ Human brain is {eq:.1f}× larger than expected!")
print(f"➜ We have {human_brain - expected:.0f}g of 'extra' brain")

print("\n✓ EQ calculator ready!")

## Part 2: Species Database

Real data from Chapter 9 and scientific literature:

In [ ]:
# Complete species database with Chapter 9 data
species_db = {
    'Human': {
        'body_kg': 70, 'brain_g': 1350, 'group': 'Mammal',
        'special': ['Language', 'Abstract reasoning', 'Cultural learning', 'Tool mastery'],
        'desc': 'Massive cortical expansion in association areas. Specialized language networks. '
                'Unmatched abstract reasoning. EQ of 7.5 is highest known.'
    },
    'Dolphin': {
        'body_kg': 160, 'brain_g': 1600, 'group': 'Mammal',
        'special': ['Echolocation', 'Social cognition', 'Communication', 'Self-awareness'],
        'desc': 'Second highest EQ (~5.3). Complex social networks. Sophisticated echolocation. '
                'Passes mirror self-recognition test.'
    },
    'Elephant': {
        'body_kg': 5000, 'brain_g': 5000, 'group': 'Mammal',
        'special': ['Social intelligence', 'Memory', 'Empathy', 'Tool use'],
        'desc': 'Largest absolute brain (5kg). Complex social structure. Mourning behavior. '
                'Exceptional memory. Despite low EQ (~2.0), highly intelligent.'
    },
    'Chimpanzee': {
        'body_kg': 50, 'brain_g': 400, 'group': 'Mammal',
        'special': ['Tool use', 'Social learning', 'Problem solving', 'Culture'],
        'desc': 'EQ ~2.3. Advanced tool manufacture. Complex hierarchies. '
                'Cultural transmission of knowledge. Mirror test positive.'
    },
    'Crow': {
        'body_kg': 0.5, 'brain_g': 10, 'group': 'Bird',
        'special': ['Tool manufacture', 'Causal reasoning', 'Planning', 'Innovation'],
        'desc': 'Despite tiny brain (10g), EQ ~1.3. Makes complex tools with hooks and barbs. '
                'Multi-step problem solving. High neuron density in pallium.'
    },
    'Parrot': {
        'body_kg': 0.4, 'brain_g': 12, 'group': 'Bird',
        'special': ['Vocal learning', 'Symbolic thinking', 'Social cognition', 'Counting'],
        'desc': 'EQ ~1.2. African greys learn 150+ words. Understand concepts like same/different. '
                'Can count and do simple math. Avian brain efficiency.'
    },
    'Dog': {
        'body_kg': 30, 'brain_g': 72, 'group': 'Mammal',
        'special': ['Social cognition', 'Human communication', 'Scent', 'Training'],
        'desc': 'EQ ~1.2. Domestication shaped brain. Exceptional at reading human cues. '
                'Enhanced olfactory processing. Social intelligence.'
    },
    'Cat': {
        'body_kg': 4.5, 'brain_g': 30, 'group': 'Mammal',
        'special': ['Hunting', 'Motor coordination', 'Memory', 'Independence'],
        'desc': 'EQ ~1.0 (average). Efficient brain organization. Excellent motor control. '
                'Good spatial memory and object permanence.'
    },
    'Rat': {
        'body_kg': 0.3, 'brain_g': 2, 'group': 'Mammal',
        'special': ['Spatial memory', 'Social learning', 'Flexibility', 'Adaptability'],
        'desc': 'EQ ~0.4. Versatile learner despite low EQ. Complex social behavior. '
                'Remarkable adaptability to environments. Good problem solvers.'
    },
    'Lizard': {
        'body_kg': 15, 'brain_g': 12, 'group': 'Reptile',
        'special': ['Spatial memory', 'Problem solving', 'Learning', 'Counting'],
        'desc': 'EQ ~0.4. Monitor lizards show highest reptile cognition. '
                'Count objects. Learn complex tasks. Some tool use reported.'
    },
    'Salmon': {
        'body_kg': 5, 'brain_g': 2, 'group': 'Fish',
        'special': ['Navigation', 'Olfaction', 'Migration', 'Homing'],
        'desc': 'EQ ~0.2. Extraordinary olfactory navigation. Return to birthplace after years. '
                'Detect one part per billion chemical signatures.'
    },
    'Salamander': {
        'body_kg': 0.02, 'brain_g': 0.05, 'group': 'Amphibian',
        'special': ['Habitat recognition', 'Conditioning', 'Regeneration', 'Dual-environment'],
        'desc': 'EQ ~0.3. Simple but effective brain. Habitat recognition. '
                'Remarkable neural regeneration capacity. Efficient for lifestyle.'
    }
}

# Calculate EQ for all
for name in species_db:
    sp = species_db[name]
    eq, _ = calculate_eq(sp['brain_g'], sp['body_kg'])
    sp['eq'] = eq

# Display by group
print("SPECIES DATABASE")
print("=" * 70)
print(f"Total species: {len(species_db)}\n")

for group in ['Mammal', 'Bird', 'Reptile', 'Fish', 'Amphibian']:
    species = [n for n, d in species_db.items() if d['group'] == group]
    if species:
        print(f"{group}s ({len(species)}):")
        for sp_name in sorted(species, key=lambda x: species_db[x]['eq'], reverse=True):
            sp = species_db[sp_name]
            print(f"  {sp_name:15} EQ: {sp['eq']:5.2f}  |  Brain: {sp['brain_g']:7.1f}g  |  Body: {sp['body_kg']:7.1f}kg")
        print()

print("✓ Database ready for analysis!")

## Part 3: Interactive EQ Calculator

In [ ]:
# Comparison data storage
comparison_data = []

def analyze_species(species_name):
    """Analyze and visualize species EQ"""
    
    if species_name not in species_db:
        print(f"Species {species_name} not found")
        return
    
    sp = species_db[species_name]
    brain = sp['brain_g']
    body = sp['body_kg']
    eq = sp['eq']
    _, expected = calculate_eq(brain, body)
    
    # Determine category
    if eq >= 5.0:
        category, color = "Exceptional", "#E74C3C"
    elif eq >= 2.0:
        category, color = "Very High", "#3498DB"
    elif eq >= 1.0:
        category, color = "Above Average", "#2ECC71"
    elif eq >= 0.5:
        category, color = "Average", "#F39C12"
    else:
        category, color = "Below Average", "#95A5A6"
    
    # Create 4-panel visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Encephalization Quotient', 'Brain vs Body Size',
                       'Actual vs Expected Brain', 'Cognitive Profile'),
        specs=[[{'type': 'indicator'}, {'type': 'scatter'}],
               [{'type': 'bar'}, {'type': 'bar'}]],
        vertical_spacing=0.15,
        horizontal_spacing=0.12
    )
    
    # 1. EQ Gauge
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=eq,
        title={'text': category, 'font': {'size': 16}},
        number={'suffix': " EQ", 'font': {'size': 24}},
        gauge={
            'axis': {'range': [0, 8], 'tickwidth': 1},
            'bar': {'color': color, 'thickness': 0.75},
            'steps': [
                {'range': [0, 0.5], 'color': '#ECF0F1'},
                {'range': [0.5, 1.0], 'color': '#D5DBDB'},
                {'range': [1.0, 2.0], 'color': '#AED6F1'},
                {'range': [2.0, 5.0], 'color': '#85C1E9'},
                {'range': [5.0, 8.0], 'color': '#FADBD8'}
            ],
            'threshold': {
                'line': {'color': "black", 'width': 2},
                'thickness': 0.75,
                'value': 1.0
            }
        }
    ), row=1, col=1)
    
    # 2. All species scatter
    for sp_name, sp_data in species_db.items():
        is_current = (sp_name == species_name)
        fig.add_trace(go.Scatter(
            x=[sp_data['body_kg']],
            y=[sp_data['brain_g']],
            mode='markers+text' if is_current else 'markers',
            name=sp_name,
            marker=dict(
                size=20 if is_current else 10,
                color=color if is_current else '#95A5A6',
                symbol='star' if is_current else 'circle',
                line=dict(width=2, color='white') if is_current else dict(width=0)
            ),
            text=[sp_name] if is_current else None,
            textposition="top center",
            showlegend=False,
            hovertemplate=f"{sp_name}<br>Body: {sp_data['body_kg']:.1f}kg<br>Brain: {sp_data['brain_g']:.1f}g<br>EQ: {sp_data['eq']:.2f}<extra></extra>"
        ), row=1, col=2)
    
    # 3. Brain comparison
    fig.add_trace(go.Bar(
        x=['Expected', 'Actual'],
        y=[expected, brain],
        marker_color=['#BDC3C7', color],
        text=[f"{expected:.0f}g", f"{brain:.0f}g"],
        textposition='auto',
        textfont=dict(size=14, color='white'),
        showlegend=False
    ), row=2, col=1)
    
    # 4. Specializations
    if len(sp['special']) > 0:
        specs = sp['special'][:4]  # Top 4
        values = [100, 85, 70, 55][:len(specs)]
        fig.add_trace(go.Bar(
            y=specs,
            x=values,
            orientation='h',
            marker_color=[color] * len(specs),
            text=specs,
            textposition='inside',
            textfont=dict(color='white', size=11),
            showlegend=False
        ), row=2, col=2)
    
    # Update axes
    fig.update_xaxes(title_text="Body Mass (kg)", type="log", row=1, col=2)
    fig.update_yaxes(title_text="Brain Mass (g)", type="log", row=1, col=2)
    fig.update_yaxes(title_text="Mass (g)", row=2, col=1)
    fig.update_xaxes(title_text="Specialization Level", row=2, col=2)
    
    fig.update_layout(
        height=700,
        title_text=f"<b>{species_name} Brain Analysis</b>",
        showlegend=False,
        font=dict(size=11)
    )
    
    # Print summary
    print("\n" + "="*70)
    print(f"ANALYSIS: {species_name}")
    print("="*70)
    print(f"Group: {sp['group']}")
    print(f"Body Mass: {body:,.1f} kg")
    print(f"Brain Mass: {brain:,.1f} g")
    print(f"Expected Brain: {expected:,.1f} g (for this body size)")
    print(f"EQ: {eq:.2f} ({category})")
    print(f"\n➜ Brain is {eq:.1f}× {'LARGER' if eq > 1 else 'smaller'} than expected")
    if eq > 1:
        print(f"➜ Has {brain - expected:.0f}g of 'extra' brain capacity")
    print(f"\nSpecializations: {', '.join(sp['special'])}")
    print(f"\n{sp['desc']}")
    print("="*70)
    
    fig.show()
    
    return {'species': species_name, 'eq': eq, 'category': category}

# Create dropdown
species_dropdown = Dropdown(
    options=sorted(species_db.keys(), key=lambda x: species_db[x]['eq'], reverse=True),
    value='Human',
    description='Species:',
    style={'description_width': '80px'},
    layout={'width': '300px'}
)

display(HTML("<h3>🧠 Interactive EQ Explorer</h3>"))
display(HTML("<p>Select a species to analyze its brain size and intelligence:</p>"))
interact(analyze_species, species_name=species_dropdown);

## Part 4: Species Comparison Tools

In [ ]:
def compare_multiple():
    """Compare all species side-by-side"""
    
    # Prepare data
    df = pd.DataFrame([
        {
            'Species': name,
            'Group': data['group'],
            'Body (kg)': data['body_kg'],
            'Brain (g)': data['brain_g'],
            'EQ': data['eq']
        }
        for name, data in species_db.items()
    ]).sort_values('EQ', ascending=False)
    
    # Create visualizations
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('EQ Ranking', 'Brain vs Body Mass',
                       'EQ by Group', 'Absolute Brain Size'),
        specs=[[{'type': 'bar'}, {'type': 'scatter'}],
               [{'type': 'box'}, {'type': 'bar'}]]
    )
    
    # 1. EQ Ranking
    colors = ['#E74C3C' if eq >= 5 else '#3498DB' if eq >= 2 else '#2ECC71' if eq >= 1 else '#F39C12' if eq >= 0.5 else '#95A5A6' 
              for eq in df['EQ']]
    
    fig.add_trace(go.Bar(
        x=df['Species'],
        y=df['EQ'],
        marker_color=colors,
        text=[f"{eq:.2f}" for eq in df['EQ']],
        textposition='outside',
        showlegend=False
    ), row=1, col=1)
    
    # 2. Brain vs Body scatter with groups
    for group in df['Group'].unique():
        group_data = df[df['Group'] == group]
        fig.add_trace(go.Scatter(
            x=group_data['Body (kg)'],
            y=group_data['Brain (g)'],
            mode='markers+text',
            name=group,
            text=group_data['Species'],
            textposition='top center',
            marker=dict(size=12),
            textfont=dict(size=9)
        ), row=1, col=2)
    
    # 3. EQ by group (box plot)
    for group in ['Mammal', 'Bird', 'Reptile', 'Fish', 'Amphibian']:
        group_data = df[df['Group'] == group]
        if len(group_data) > 0:
            fig.add_trace(go.Box(
                y=group_data['EQ'],
                name=group,
                boxmean='sd',
                showlegend=False
            ), row=2, col=1)
    
    # 4. Absolute brain size
    df_brain = df.nlargest(8, 'Brain (g)')
    fig.add_trace(go.Bar(
        x=df_brain['Species'],
        y=df_brain['Brain (g)'],
        marker_color='#9B59B6',
        text=[f"{b:.0f}g" for b in df_brain['Brain (g)']],
        textposition='outside',
        showlegend=False
    ), row=2, col=2)
    
    # Update layout
    fig.update_xaxes(tickangle=45, row=1, col=1)
    fig.update_xaxes(title_text="Body Mass (kg)", type="log", row=1, col=2)
    fig.update_yaxes(title_text="Brain Mass (g)", type="log", row=1, col=2)
    fig.update_yaxes(title_text="EQ", row=1, col=1)
    fig.update_yaxes(title_text="EQ", row=2, col=1)
    fig.update_yaxes(title_text="Brain Mass (g)", row=2, col=2)
    fig.update_xaxes(tickangle=45, row=2, col=2)
    
    fig.update_layout(
        height=800,
        title_text="<b>Complete Species Comparison</b>",
        showlegend=True
    )
    
    # Print summary
    print("\nCOMPLETE SPECIES RANKING")
    print("="*70)
    print(f"{'Rank':<6}{'Species':<18}{'Group':<12}{'EQ':<8}{'Brain':<12}{'Body':<10}")
    print("="*70)
    for idx, row in df.iterrows():
        rank = list(df.index).index(idx) + 1
        print(f"{rank:<6}{row['Species']:<18}{row['Group']:<12}{row['EQ']:<8.2f}"
              f"{row['Brain (g)']:<12.1f}{row['Body (kg)']:<10.1f}")
    print("="*70)
    
    fig.show()
    
    return df

# Button to trigger comparison
compare_btn = Button(
    description='📊 Compare All Species',
    button_style='info',
    layout={'width': '200px'},
    icon='bar-chart'
)

output_area = Output()

def on_compare_click(b):
    with output_area:
        clear_output(wait=True)
        compare_multiple()

compare_btn.on_click(on_compare_click)

display(HTML("<h3>📊 Multi-Species Comparison</h3>"))
display(compare_btn)
display(output_area)

## Part 5: Challenge Problems

Apply your understanding to real-world scenarios:

### Challenge 1: The Crow Paradox 🦅

**Question**: New Caledonian crows have tiny brains (10g) but can solve problems that stump animals with 100× larger brains. How is this possible?

Use the calculator to compare:
- Crow: 0.5 kg body, 10g brain
- Dog: 30 kg body, 72g brain

**Analysis**:
1. Calculate both EQs
2. Compare absolute brain sizes
3. What does this tell you about brain organization vs size?

<details>
<summary>Hint</summary>
Consider: neuron density (birds have 2× more neurons per gram than mammals!), brain organization (avian pallium vs mammalian cortex), and metabolic efficiency.
</details>

<details>
<summary>Solution</summary>

**Crow EQ**: ~1.3 (above average)  
**Dog EQ**: ~1.2 (similar!)

**Key insights**:
1. **Similar relative brain size** despite 7× size difference
2. **Avian advantage**: Birds pack 2× more neurons per cubic mm
3. **Nuclear organization**: Bird pallium processes info differently than mammalian cortex
4. **Efficiency**: Crow's 10g brain has ~1.5 billion neurons (similar to some small primates!)
5. **Specialization**: Crow brain optimized for visual-spatial problem solving

**Lesson**: *Absolute size matters less than organization and neuron density!*
</details>

In [ ]:
# Challenge 1 workspace
print("CHALLENGE 1: THE CROW PARADOX")
print("="*70)

# Compare crow and dog
crow = species_db['Crow']
dog = species_db['Dog']

print("\nCROW:")
print(f"  Brain: {crow['brain_g']:.1f}g")
print(f"  Body: {crow['body_kg']:.1f}kg")
print(f"  EQ: {crow['eq']:.2f}")

print("\nDOG:")
print(f"  Brain: {dog['brain_g']:.1f}g")
print(f"  Body: {dog['body_kg']:.1f}kg")
print(f"  EQ: {dog['eq']:.2f}")

print(f"\nBrain size ratio: Dog has {dog['brain_g']/crow['brain_g']:.1f}× more brain tissue")
print(f"BUT... EQ ratio: Only {dog['eq']/crow['eq']:.2f}× relative difference")
print(f"\nEstimated neurons: Crow ~1.5 billion, Dog ~2.2 billion")
print(f"Neuron density: Crow has 2× more neurons per gram!")

print("\n" + "="*70)
print("YOUR ANALYSIS:")
print("-"*70)
print("Why can crows match or exceed dogs in problem-solving?")
print("What does this teach us about intelligence?")
print("="*70)

### Challenge 2: Designing an Alien Brain 👽

**Scenario**: You're designing an intelligent alien for a sci-fi movie. The alien:
- Lives on a high-gravity planet (body must be small and strong)
- Needs human-level intelligence
- Body mass: 20 kg (compact, muscular)

**Questions**:
1. What brain mass would give it EQ = 7.5 (human level)?
2. Is this realistic? (Hint: check expected brain for 20kg body)
3. What if you only need EQ = 2.0 (chimp level)?

<details>
<summary>Hint</summary>
Use the formula: Brain mass = EQ × Expected brain  
Where Expected = 0.12 × 20^0.67 × 1000 (in grams)
</details>

In [ ]:
# Challenge 2 workspace
print("CHALLENGE 2: ALIEN BRAIN DESIGN")
print("="*70)

alien_body = 20  # kg
target_eqs = [7.5, 5.0, 2.0, 1.0]

# Calculate expected brain
expected_kg = 0.12 * (alien_body ** 0.67)
expected_g = expected_kg * 1000

print(f"Alien body mass: {alien_body} kg")
print(f"Expected brain (EQ=1.0): {expected_g:.1f}g\n")

print("DESIGN OPTIONS:")
print("-"*70)
for target_eq in target_eqs:
    required_brain = expected_g * target_eq
    
    # Find comparable Earth species
    comparable = None
    for name, sp in species_db.items():
        if abs(sp['eq'] - target_eq) < 0.3:
            comparable = name
            break
    
    # Feasibility check
    brain_body_ratio = (required_brain / 1000) / alien_body * 100
    if brain_body_ratio > 3:
        feasible = "⚠️  HIGH (metabolically expensive)"
    elif brain_body_ratio > 2:
        feasible = "✓ Feasible (like primates)"
    else:
        feasible = "✓ Very feasible"
    
    print(f"\nTarget EQ {target_eq}:")
    print(f"  Required brain: {required_brain:.0f}g")
    print(f"  Brain/body ratio: {brain_body_ratio:.1f}%")
    print(f"  Comparable to: {comparable if comparable else 'No direct match'}")
    print(f"  Feasibility: {feasible}")

print("\n" + "="*70)
print("YOUR DESIGN CHOICE:")
print("-"*70)
print("Which EQ would you choose? Why?")
print("What trade-offs are involved?")
print("="*70)

### Challenge 3: Evolution on Earth 2.0 🌍

**Scenario**: Scientists discover Earth 2.0 with similar conditions but slightly lower gravity (0.8× Earth).

**Question**: How might this affect brain evolution?

Consider:
- Lower gravity = less energy needed for support
- Could animals afford larger brains?
- Would body size increase or decrease?
- How would this affect EQ trends?

**Predict**: If humans evolved on Earth 2.0 with same brain (1350g) but could have 20% larger bodies due to lower gravity (84kg vs 70kg), what would human EQ be?

<details>
<summary>Solution</summary>

Earth 1.0 Human: EQ = 7.5  
Earth 2.0 Human (84kg body, 1350g brain): EQ ≈ 6.8

**Insights**:
1. Larger bodies need more brain just for control
2. EQ would actually *decrease* if brain size stayed same
3. To maintain EQ=7.5, brain would need to be ~1500g
4. **Prediction**: Lower gravity might favor even LARGER brains (less neck strain!)
5. Could see species with EQ > 10!
</details>

In [ ]:
# Challenge 3 workspace
print("CHALLENGE 3: EARTH 2.0 EVOLUTION")
print("="*70)

# Earth 1.0 human
earth1_body = 70
earth1_brain = 1350
earth1_eq, earth1_exp = calculate_eq(earth1_brain, earth1_body)

# Earth 2.0 scenario: 20% heavier bodies (lower gravity, less costly)
earth2_body = earth1_body * 1.2  # 84 kg
earth2_brain_same = earth1_brain  # Keep brain same
earth2_eq_same, earth2_exp = calculate_eq(earth2_brain_same, earth2_body)

# What brain size needed to maintain EQ?
earth2_brain_needed = earth2_exp * earth1_eq

print("EARTH 1.0 (current):")
print(f"  Body: {earth1_body}kg, Brain: {earth1_brain}g")
print(f"  EQ: {earth1_eq:.2f}\n")

print("EARTH 2.0 SCENARIO A (same brain, bigger body):")
print(f"  Body: {earth2_body:.0f}kg, Brain: {earth2_brain_same}g")
print(f"  EQ: {earth2_eq_same:.2f} (↓ from {earth1_eq:.2f})")
print(f"  Result: Intelligence DECREASES relative to body\n")

print("EARTH 2.0 SCENARIO B (maintain same EQ):")
print(f"  Body: {earth2_body:.0f}kg, Brain needed: {earth2_brain_needed:.0f}g")
print(f"  That's {earth2_brain_needed - earth1_brain:.0f}g MORE brain!")
print(f"  Result: Maintain intelligence level\n")

print("EARTH 2.0 SCENARIO C (even SMARTER):")
earth2_super_brain = earth2_brain_needed * 1.2
earth2_super_eq, _ = calculate_eq(earth2_super_brain, earth2_body)
print(f"  Body: {earth2_body:.0f}kg, Brain: {earth2_super_brain:.0f}g")
print(f"  EQ: {earth2_super_eq:.2f} (↑ from {earth1_eq:.2f})")
print(f"  Result: SUPER-INTELLIGENCE!\n")

print("="*70)
print("ANALYSIS:")
print("-"*70)
print("• Lower gravity allows larger bodies without structural cost")
print("• But larger bodies need more brain just for control")
print("• Less neck strain = could support even larger brains")
print("• Prediction: EQ could exceed 10.0 on low-gravity worlds!")
print("="*70)

## Part 6: Export Your Work

In [ ]:
def export_results():
    """Export lab results"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/content"  # Google Colab path
    
    # Create summary data
    data = []
    for name, sp in sorted(species_db.items(), key=lambda x: x[1]['eq'], reverse=True):
        data.append({
            'Species': name,
            'Group': sp['group'],
            'Body_kg': sp['body_kg'],
            'Brain_g': sp['brain_g'],
            'EQ': round(sp['eq'], 2),
            'Specializations': ', '.join(sp['special'])
        })
    
    df = pd.DataFrame(data)
    
    # Save CSV
    csv_file = f"{output_dir}/lab_9_1_results_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Data saved: {csv_file}")
    
    # Create summary visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('EQ Rankings', 'Brain vs Body Mass'),
        specs=[[{'type': 'bar'}, {'type': 'scatter'}]]
    )
    
    # EQ bar chart
    colors = ['#E74C3C' if eq >= 5 else '#3498DB' if eq >= 2 else '#2ECC71' if eq >= 1 else '#F39C12' 
              for eq in df['EQ']]
    
    fig.add_trace(go.Bar(
        x=df['Species'],
        y=df['EQ'],
        marker_color=colors,
        text=df['EQ'],
        textposition='outside',
        showlegend=False
    ), row=1, col=1)
    
    # Scatter plot
    for group in df['Group'].unique():
        group_data = df[df['Group'] == group]
        fig.add_trace(go.Scatter(
            x=group_data['Body_kg'],
            y=group_data['Brain_g'],
            mode='markers+text',
            name=group,
            text=group_data['Species'],
            textposition='top center',
            marker=dict(size=12)
        ), row=1, col=2)
    
    fig.update_xaxes(tickangle=45, row=1, col=1)
    fig.update_xaxes(title_text="Body Mass (kg)", type="log", row=1, col=2)
    fig.update_yaxes(title_text="EQ", row=1, col=1)
    fig.update_yaxes(title_text="Brain Mass (g)", type="log", row=1, col=2)
    
    fig.update_layout(
        height=500,
        title_text='<b>Lab 9.1 Summary: Encephalization Quotient Analysis</b>',
        showlegend=True
    )
    
    # Save HTML
    html_file = f"{output_dir}/lab_9_1_summary_{timestamp}.html"
    fig.write_html(html_file)
    print(f"✓ Visualization saved: {html_file}")
    
    # Try PNG (optional)
    try:
        png_file = f"{output_dir}/lab_9_1_summary_{timestamp}.png"
        fig.write_image(png_file, width=1200, height=600)
        print(f"✓ PNG saved: {png_file}")
    except:
        print("ℹ PNG export requires kaleido (optional)")
    
    print("\n" + "="*60)
    print("EXPORT COMPLETE!")
    print("="*60)
    print(f"Files saved to: {output_dir}")
    print("\nTo download:")
    print("1. Click the folder icon 📁 on the left")
    print("2. Files are in the main /content folder (at the top level)")
    print("3. Right-click and select 'Download'")
    print("\n✓ Your results are ready!")

# Export button
export_btn = Button(
    description='📥 Export Results',
    button_style='success',
    icon='download',
    layout={'width': '200px'}
)

export_btn.on_click(lambda b: export_results())

display(HTML("<h3>📤 Export Your Work</h3>"))
display(HTML("<p>Click below to save your EQ analysis data and visualizations:</p>"))
display(export_btn)

## Summary & Reflection

### What You've Learned

✅ **Encephalization Quotient (EQ)** - A better measure than absolute brain size  
✅ **Brain-body relationships** - Allometric scaling across vertebrates  
✅ **Relative vs absolute size** - Why crows are smarter than expected  
✅ **Evolutionary patterns** - Mammals and birds show convergent high EQ  
✅ **Brain organization** - Structure matters as much as size  

### Key Insights

**From the data:**
- **Humans**: EQ 7.5 (exceptional)
- **Dolphins**: EQ 5.3 (very high)
- **Birds**: High EQ despite small absolute size
- **Large mammals**: Lower EQ but more absolute processing power

**Pattern recognition:**
1. EQ correlates with cognitive flexibility
2. But not a perfect predictor (elephants have low EQ, high intelligence)
3. Brain organization and neuron density matter enormously
4. Multiple evolutionary pathways to intelligence

### Connection to Chapter 9

This lab illustrated core concepts from **Section 9.3**:
- Allometric scaling of brain size
- Regional brain specialization
- Avian vs mammalian brain organization
- Evolution of cognitive capabilities

### Real-World Applications

- **Neuroscience**: Understanding intelligence evolution
- **AI**: Lessons for artificial intelligence design
- **Conservation**: Predicting cognitive needs of species
- **Astrobiology**: Predicting alien intelligence
- **Medicine**: Understanding neurodevelopment

### The Big Picture

Intelligence isn't just about size - it's about:
- **Efficiency**: Neuron density and organization
- **Specialization**: What the brain is optimized for
- **Trade-offs**: Energy cost vs cognitive benefit
- **Evolution**: Multiple pathways to smart solutions

**Congratulations on completing Lab 9.1!** 🎉

You now understand why a crow with a walnut-sized brain can outsmart animals with brains 100× larger!